In [2]:
#Test Spark to make sure it works delete later.
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext('local[*]')
# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)
sc.stop()


In [20]:
from pyspark.sql import SQLContext
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import Tokenizer
#sc = pyspark.SparkContext('local[*]')
sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('160558090672531_facebook_statuses.csv')
df.show()
df = df.na.drop(subset=["status_message"])
messages = df.select("status_message")

tokenizer = Tokenizer(inputCol="status_message", outputCol="filtered")
filterw = tokenizer.transform(messages)
filterw.show()
remover = StopWordsRemover(inputCol="filtered", outputCol="filtered1")
filtered_final = remover.transform(filterw)
filtered_final.show()
messages = filtered_final.select("filtered1")
message_rdd=messages.rdd


+--------------------+--------------------+----------+-----------+--------------------+-------------------+-------------+------------+----------+---------+---------+--------+---------+--------+----------+
|           status_id|      status_message| link_name|status_type|         status_link|   status_published|num_reactions|num_comments|num_shares|num_likes|num_loves|num_wows|num_hahas|num_sads|num_angrys|
+--------------------+--------------------+----------+-----------+--------------------+-------------------+-------------+------------+----------+---------+---------+--------+---------+--------+----------+
|160558090672531_1...|So, rain is in th...|      null|     status|                null|2016-12-22 13:34:30|            4|          15|         0|        4|        0|       0|        0|       0|         0|
|160558090672531_1...|                null|      null|     status|                null|2016-12-22 21:38:02|           15|           3|         0|       14|        0|       1|      

In [23]:
statuses = message_rdd.flatMap(lambda x: x)
classic = statuses.map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
print(classic.collectAsMap())


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 81.0 failed 1 times, most recent failure: Lost task 0.0 in stage 81.0 (TID 98, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\img\Documents\spark-2.0.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 172, in main
  File "C:\Users\img\Documents\spark-2.0.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 167, in process
  File "C:\Users\img\Documents\spark-2.0.2-bin-hadoop2.7\python\pyspark\rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\Users\img\Documents\spark-2.0.2-bin-hadoop2.7\python\pyspark\rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\Users\img\Documents\spark-2.0.2-bin-hadoop2.7\python\pyspark\rdd.py", line 317,

In [1]:
#Irrelvant
import nltk
from nltk.corpus import stopwords  
import re
import sys
from operator import add
import string 

def word_tokenize(x):
    return nltk.word_tokenize(x)

    return x 
def shit(word):
    l = word_tokenize(word)
    return filtered_words
def garbage_removal(y):
    print("hello this is" + y)
    return y
    #if y not in stopwords.words('english') and y not in string.punctutation:
        #return y
   
    
    #eturn x.replace("""([\p{Punct}&&[^.]]|\b\p{IsLetter}{1,2}\b)\s*""", "")



     
statuses = message_rdd.flatMap(lambda x: x)
statuses = statuses.filter(lambda x: x is not None)
print(statuses.take(5))
words = statuses.map(lambda x: shit(x))
words2 = words.flatMap(lambda x: x)

print(words2.take(5))

filtered_words.take(5)

print(filtered_words.take(5))


sc.stop()

NameError: name 'message_rdd' is not defined